In [2]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('bengaluru_restaurants.db')
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE customer_details (
               id INTEGER PRIMARY KEY AUTOINCREMENT,
               name TEXT,
               phone_number TEXT,
               email TEXT)
'''
)

In [ ]:
import sqlite3
import csv
import random

# Connect to the database
conn = sqlite3.connect('bengaluru_restaurants.db')
cursor = conn.cursor()

# Drop existing table if present
cursor.execute("DROP TABLE IF EXISTS restaurant_locations")

# Create the table (without restaurant name)
cursor.execute('''
CREATE TABLE restaurant_locations (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    cuisine_type TEXT,
    address TEXT NOT NULL,
    area TEXT UNIQUE,
    pincode TEXT,
    phone_number TEXT,
    opening_hours TEXT,
    veg_type TEXT CHECK(veg_type IN ('Veg', 'Non-Veg', 'Both')),
    seats_available INTEGER DEFAULT 0,
    booking_status TEXT DEFAULT 'Empty',
    nearby_locations TEXT
)
''')

# Define 20 unique locations
areas = [
    'Indiranagar', 'Koramangala', 'HSR Layout', 'Jayanagar', 'BTM Layout',
    'Whitefield', 'Hebbal', 'Malleshwaram', 'Rajajinagar', 'Banashankari',
    'Ulsoor', 'Marathahalli', 'Yelahanka', 'Sarjapur', 'Bellandur',
    'Frazer Town', 'RT Nagar', 'Kengeri', 'Nagarbhavi', 'Vijayanagar'
]

veg_options = ['Veg', 'Non-Veg', 'Both']
cuisine_options = ['Veg', 'Non-Veg', 'Both']

# Insert 1 restaurant per location
records = []
for i, area in enumerate(areas):
    cuisine_type = random.choice(cuisine_options)
    address = f"{100 + i*5} Example Street"
    pincode = f"5600{i:02d}"
    phone_number = f"080-4000-{1000 + i}"
    opening_hours = "10:00 AM - 10:00 PM"
    veg_type = random.choice(veg_options)
    seats = 0 if i % 5 == 0 else random.randint(20, 60)  # Set some to 0

    records.append((cuisine_type, address, area, pincode, phone_number, opening_hours, veg_type, seats))

# Bulk insert
cursor.executemany('''
    INSERT INTO restaurant_locations
    (cuisine_type, address, area, pincode, phone_number, opening_hours, veg_type, seats_available)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
''', records)

# Update booking_status based on seats_available
cursor.execute('''
    UPDATE restaurant_locations
    SET booking_status = CASE
        WHEN seats_available = 0 THEN 'Booked'
        ELSE 'Empty'
    END
''')

# Assign 1–2 random nearby locations
for area in areas:
    nearby = random.sample([a for a in areas if a != area], k=random.choice([1, 2]))
    nearby_str = ', '.join(nearby)
    cursor.execute('''
        UPDATE restaurant_locations
        SET nearby_locations = ?
        WHERE area = ?
    ''', (nearby_str, area))

# Commit changes
conn.commit()

# Export to CSV



✅ Exported 20 unique restaurant locations to 'restaurant_locations_export.csv'


In [44]:
# Connect to the database
conn = sqlite3.connect('bengaluru_restaurants.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM restaurant_locations")
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]

with open('restaurant_locations_export.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(columns)
    writer.writerows(rows)

print("✅ Exported 20 unique restaurant locations to 'restaurant_locations_export.csv'")

# Close DB connection
conn.close()

✅ Exported 20 unique restaurant locations to 'restaurant_locations_export.csv'
